<a href="https://colab.research.google.com/github/refecher/rockets-web-scraping/blob/master/Space_Missions_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Exploration Analysis on Space Missions Since 1957

<center><img src=https://img-c.udemycdn.com/redactor/raw/assignment/2020-11-01_16-57-32-41f2067132bc0f59fba67ec69e75e916.jpg height=500>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Missions in 1957!</center>

# 1. Imports

### Install Package that Contains Country Codes

In [ ]:
%pip install iso3166

### Upgrade Plotly Package

In [ ]:
%pip install --upgrade plotly

     |████████████████████████████████| 15.2 MB 9.2 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


### Import Statements

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from iso3166 import countries

import matplotlib.dates as mdates

# 2. Preliminary Data Exploration 🔎

### Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

### Load and Display the Data
As seen below, the data contains 6315 rows and 7 columns. In this first look, we can see some NaN values in the price column. Let's check if there's more NaN values in the other columns.

In [ ]:
data = pd.read_csv('output.csv', index_col=0)
data

,organisation,location,date,details,price_in_million,mission_status,rocket_status
0,SpaceX,"SLC-40, Cape Canaveral SFS, Florida, USA","Sat May 14, 2022 3:40 PM CDT",Falcon 9 Block 5 | Starlink Group 4-15,$67.0 million,Success,Active
1,SpaceX,"SLC-4E, Vandenberg SFB, California, USA","Fri May 13, 2022 5:07 PM CDT",Falcon 9 Block 5 | Starlink Group 4-13,$67.0 million,Success,Active
2,i-Space,"Site 95, Jiuquan Satellite Launch Center, China","Fri May 13, 2022 2:09 AM CDT",Hyperbola-1 | Jilin-1 Mofang-01A(R),NaN,Failure,Active
3,CASC,"LC-201, Wenchang Satellite Launch Center, China","Mon May 9, 2022 12:56 PM CDT",Long March 7 | Tianzhou 4,NaN,Success,Active
4,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri May 6, 2022 4:42 AM CDT",Falcon 9 Block 5 | Starlink Group 4-17,$67.0 million,Success,Active
...,...,...,...,...,...,...,...
6310,US Navy,"LC-18A, Cape Canaveral SFS, Florida, USA","Wed Feb 5, 1958 1:33 AM GMT-6",Vanguard | Vanguard TV3BU,NaN,Failure,Retired
6311,ABMA,"LC-26A, Cape Canaveral SFS, Florida, USA","Fri Jan 31, 1958 9:47 PM GMT-6",Juno I | Explorer 1,NaN,Success,Retired
6312,US Navy,"LC-18A, Cape Canaveral SFS, Florida, USA","Fri Dec 6, 1957 10:44 AM GMT-6",Vanguard | Vanguard TV3,NaN,Failure,Retired
6313,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sat Nov 2, 1957 8:30 PM GMT-6",Sputnik 8K71PS | Sputnik 2,NaN,Success,Retired


###Gathering General Information
The table below shows us that we have a total of 7 columns and only one of the columns has missing values.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6315 entries, 0 to 6314
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   organisation      6315 non-null   object
 1   location          6315 non-null   object
 2   date              6315 non-null   object
 3   details           6315 non-null   object
 4   price_in_million  2325 non-null   object
 5   mission_status    6315 non-null   object
 6   rocket_status     6315 non-null   object
dtypes: object(7)
memory usage: 394.7+ KB


###Check for Nulls and Duplicates
The database has no duplicate rows, but there are a total of 3990 Null values in one column, which is more than 60% of the total values for that column missing.

In [ ]:
print(f'Are there any NaN values?  {data.isnull().values.any()}')
print(f'Are there any duplicate values?  {data.duplicated().values.any()}')

Are there any NaN values?  True
Are there any duplicate values?  False


In [ ]:
percent_missing = (data.isnull().sum() * 100) / len(data)
percent_missing_df = pd.DataFrame({'column': data.columns,
                                 'percent': percent_missing})
percent_missing_df.reset_index(drop=True, inplace=True)
percent_missing_df = percent_missing_df[percent_missing_df['percent']>0]
percent_missing_df

,column,percent
4,price_in_million,63.18


# 3. Data Cleaning 🧹

###Convert 'Date' Column and Create 'Year' Column
I perform data preprocessing by conveting the 'date' column to a datetime object. After, from this column, I extract a 'year' column.

In [ ]:
data['date'] = pd.to_datetime(data['date'], utc=True)
data

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,organisation,location,date,details,price_in_million,mission_status,rocket_status
0,SpaceX,"SLC-40, Cape Canaveral SFS, Florida, USA",2022-05-14 15:40:00+00:00,Falcon 9 Block 5 | Starlink Group 4-15,$67.0 million,Success,Active
1,SpaceX,"SLC-4E, Vandenberg SFB, California, USA",2022-05-13 17:07:00+00:00,Falcon 9 Block 5 | Starlink Group 4-13,$67.0 million,Success,Active
2,i-Space,"Site 95, Jiuquan Satellite Launch Center, China",2022-05-13 02:09:00+00:00,Hyperbola-1 | Jilin-1 Mofang-01A(R),NaN,Failure,Active
3,CASC,"LC-201, Wenchang Satellite Launch Center, China",2022-05-09 12:56:00+00:00,Long March 7 | Tianzhou 4,NaN,Success,Active
4,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2022-05-06 04:42:00+00:00,Falcon 9 Block 5 | Starlink Group 4-17,$67.0 million,Success,Active
...,...,...,...,...,...,...,...
6310,US Navy,"LC-18A, Cape Canaveral SFS, Florida, USA",1958-02-04 19:33:00+00:00,Vanguard | Vanguard TV3BU,NaN,Failure,Retired
6311,ABMA,"LC-26A, Cape Canaveral SFS, Florida, USA",1958-01-31 15:47:00+00:00,Juno I | Explorer 1,NaN,Success,Retired
6312,US Navy,"LC-18A, Cape Canaveral SFS, Florida, USA",1957-12-06 04:44:00+00:00,Vanguard | Vanguard TV3,NaN,Failure,Retired
6313,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-02 14:30:00+00:00,Sputnik 8K71PS | Sputnik 2,NaN,Success,Retired


In [ ]:
data['year'] = pd.DatetimeIndex(data['date']).year
data

,organisation,location,date,details,price_in_million,mission_status,rocket_status,year
0,SpaceX,"SLC-40, Cape Canaveral SFS, Florida, USA",2022-05-14 15:40:00+00:00,Falcon 9 Block 5 | Starlink Group 4-15,$67.0 million,Success,Active,2022
1,SpaceX,"SLC-4E, Vandenberg SFB, California, USA",2022-05-13 17:07:00+00:00,Falcon 9 Block 5 | Starlink Group 4-13,$67.0 million,Success,Active,2022
2,i-Space,"Site 95, Jiuquan Satellite Launch Center, China",2022-05-13 02:09:00+00:00,Hyperbola-1 | Jilin-1 Mofang-01A(R),NaN,Failure,Active,2022
3,CASC,"LC-201, Wenchang Satellite Launch Center, China",2022-05-09 12:56:00+00:00,Long March 7 | Tianzhou 4,NaN,Success,Active,2022
4,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2022-05-06 04:42:00+00:00,Falcon 9 Block 5 | Starlink Group 4-17,$67.0 million,Success,Active,2022
...,...,...,...,...,...,...,...,...
6310,US Navy,"LC-18A, Cape Canaveral SFS, Florida, USA",1958-02-04 19:33:00+00:00,Vanguard | Vanguard TV3BU,NaN,Failure,Retired,1958
6311,ABMA,"LC-26A, Cape Canaveral SFS, Florida, USA",1958-01-31 15:47:00+00:00,Juno I | Explorer 1,NaN,Success,Retired,1958
6312,US Navy,"LC-18A, Cape Canaveral SFS, Florida, USA",1957-12-06 04:44:00+00:00,Vanguard | Vanguard TV3,NaN,Failure,Retired,1957
6313,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-02 14:30:00+00:00,Sputnik 8K71PS | Sputnik 2,NaN,Success,Retired,1957


###Create 'Country' Column and Substitute Wrong Values
Here, I extract the 'country' from the 'location' column. By doing that, I realize that some countries are wrong, so I fix that by replacing it for the correct country name.

In [ ]:
data[['location', 'country']] = data['location'].str.rsplit(', ', 1, expand=True)
data

,organisation,location,date,details,price_in_million,mission_status,rocket_status,year,country
0,SpaceX,"SLC-40, Cape Canaveral SFS, Florida",2022-05-14 15:40:00+00:00,Falcon 9 Block 5 | Starlink Group 4-15,$67.0 million,Success,Active,2022,USA
1,SpaceX,"SLC-4E, Vandenberg SFB, California",2022-05-13 17:07:00+00:00,Falcon 9 Block 5 | Starlink Group 4-13,$67.0 million,Success,Active,2022,USA
2,i-Space,"Site 95, Jiuquan Satellite Launch Center",2022-05-13 02:09:00+00:00,Hyperbola-1 | Jilin-1 Mofang-01A(R),NaN,Failure,Active,2022,China
3,CASC,"LC-201, Wenchang Satellite Launch Center",2022-05-09 12:56:00+00:00,Long March 7 | Tianzhou 4,NaN,Success,Active,2022,China
4,SpaceX,"LC-39A, Kennedy Space Center, Florida",2022-05-06 04:42:00+00:00,Falcon 9 Block 5 | Starlink Group 4-17,$67.0 million,Success,Active,2022,USA
...,...,...,...,...,...,...,...,...,...
6310,US Navy,"LC-18A, Cape Canaveral SFS, Florida",1958-02-04 19:33:00+00:00,Vanguard | Vanguard TV3BU,NaN,Failure,Retired,1958,USA
6311,ABMA,"LC-26A, Cape Canaveral SFS, Florida",1958-01-31 15:47:00+00:00,Juno I | Explorer 1,NaN,Success,Retired,1958,USA
6312,US Navy,"LC-18A, Cape Canaveral SFS, Florida",1957-12-06 04:44:00+00:00,Vanguard | Vanguard TV3,NaN,Failure,Retired,1957,USA
6313,RVSN USSR,"Site 1/5, Baikonur Cosmodrome",1957-11-02 14:30:00+00:00,Sputnik 8K71PS | Sputnik 2,NaN,Success,Retired,1957,Kazakhstan


In [ ]:
data.country.unique()

array(['USA', 'China', 'New Zealand', 'Yellow Sea', 'Russia',
       'Kazakhstan', 'Iran', 'India', 'France', 'Japan', 'South Korea',
       'Israel', 'North Korea', 'Pacific Missile Range Facility',
       'Pacific Ocean', 'Marshall Islands', 'Barents Sea', 'Brazil',
       'Gran Canaria', 'Kenya', 'Australia'], dtype=object)

In [ ]:
countries_dict = {
    'Russia' : 'Russian Federation',
    'Barents Sea': 'Russian Federation',
    'Yellow Sea': 'China',
    'Shahrud Missile Test Site': 'Iran',
    'Pacific Missile Range Facility': 'USA',
    'New Mexico' : 'USA',
    'Gran Canaria': 'USA'
}

data['country'] = data['country'].replace(countries_dict)

In [ ]:
data.country.unique()

array(['USA', 'China', 'New Zealand', 'Russian Federation', 'Kazakhstan',
       'Iran', 'India', 'France', 'Japan', 'South Korea', 'Israel',
       'North Korea', 'Pacific Ocean', 'Marshall Islands', 'Brazil',
       'Kenya', 'Australia'], dtype=object)

### Clean Price Column
Here, I decided that the price column would be better if it contained only numbers, so I removed all the unecessary characteres and multiplied the numbers to get the price of the rocket.

In [ ]:
data['price_in_million'] = data['price_in_million'].astype(str).str.replace(r'\D+', '')
data['price_in_million'] = pd.to_numeric(data['price_in_million']).apply(lambda x: x*100000)
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,organisation,location,date,details,price_in_million,mission_status,rocket_status,year,country
0,SpaceX,"SLC-40, Cape Canaveral SFS, Florida",2022-05-14 15:40:00+00:00,Falcon 9 Block 5 | Starlink Group 4-15,"67,000,000.00",Success,Active,2022,USA
1,SpaceX,"SLC-4E, Vandenberg SFB, California",2022-05-13 17:07:00+00:00,Falcon 9 Block 5 | Starlink Group 4-13,"67,000,000.00",Success,Active,2022,USA
2,i-Space,"Site 95, Jiuquan Satellite Launch Center",2022-05-13 02:09:00+00:00,Hyperbola-1 | Jilin-1 Mofang-01A(R),NaN,Failure,Active,2022,China
3,CASC,"LC-201, Wenchang Satellite Launch Center",2022-05-09 12:56:00+00:00,Long March 7 | Tianzhou 4,NaN,Success,Active,2022,China
4,SpaceX,"LC-39A, Kennedy Space Center, Florida",2022-05-06 04:42:00+00:00,Falcon 9 Block 5 | Starlink Group 4-17,"67,000,000.00",Success,Active,2022,USA
...,...,...,...,...,...,...,...,...,...
6310,US Navy,"LC-18A, Cape Canaveral SFS, Florida",1958-02-04 19:33:00+00:00,Vanguard | Vanguard TV3BU,NaN,Failure,Retired,1958,USA
6311,ABMA,"LC-26A, Cape Canaveral SFS, Florida",1958-01-31 15:47:00+00:00,Juno I | Explorer 1,NaN,Success,Retired,1958,USA
6312,US Navy,"LC-18A, Cape Canaveral SFS, Florida",1957-12-06 04:44:00+00:00,Vanguard | Vanguard TV3,NaN,Failure,Retired,1957,USA
6313,RVSN USSR,"Site 1/5, Baikonur Cosmodrome",1957-11-02 14:30:00+00:00,Sputnik 8K71PS | Sputnik 2,NaN,Success,Retired,1957,Kazakhstan


# 4. Data Analysis 🤯

In [ ]:
def plot_graph(dataframe, title, x_title):
  graph = px.bar(x = dataframe.index,
                 y = dataframe.values,
                 title=title,
                 hover_name=dataframe.index,
                 color=dataframe.values,
                 color_continuous_scale=px.colors.qualitative.Pastel)
  graph.update_layout(xaxis_title=x_title,
                    yaxis_title='Number of Rockets',
                    coloraxis_showscale=False)
  graph.show()


###Data Visualization (Year)

The graph below represent the number of launches per year from 1957 to 2022. 

2020, 1967 and 1966 are the years with the highest number of rockets launched with 157, 140, and 136 total of launches for these years.

In [ ]:
year = data['year'].value_counts()
plot_graph(year, 'Number of Rockets Launched by Year', 'Year')

###Data Visualization (Country)

The graph below represent the number of launches per country over the years. 

Russian Federation and the USA are the countries with the biggest number of rockets launched since 1957.

In [ ]:
country = data['country'].value_counts()
plot_graph(country, 'Number of Rockets Launched by Country', 'Country')

###Data Visualization (Organisation)
The organisation with the biggest number of rockets launched is the RVSN USSR. This organisation was created as a part of the Soviet Armed Forces in 1959. After the Soviet Union collapsed in 1991, their launches are attributed to Russian Federation and Kazakhstan.

In [ ]:
organization = data['organisation'].value_counts()
plot_graph(organization, 'Number of Rockets Launched by Organisation', 'Organisation')

# 4. Distribution of Missions and Rockets

In [ ]:
def plot_pie(dataframe, target_column, title):
  pie_df = dataframe[target_column].value_counts()
  graph = px.pie(
      pie_df,
      values=pie_df.values,
      names=pie_df.index,
      title=title,
      color_discrete_sequence=px.colors.qualitative.Pastel
  )
  graph.update_traces(textinfo='percent+label')
  graph.show()

### Status of Rockets Launched
Analyzing the graph below, we see that more than 80% of the rockets launched have been retired.

In [ ]:
plot_pie(data, 'rocket_status', 'Status of Rockets Launched')

### Status of Missions
More than 90% of the missions were successfull.

In [ ]:
plot_pie(data, 'mission_status', 'Status of Missions')

# 5. Number of Launches by Country
To plot the graph, we first need to get the code for each country. 

The launches made by Sea Launch weren't attributed to any country because Sea Launch is a consortium of four companies from Norway, Russia, Ukraine and the United States. It was formed to launch the Zenit family of rockets from the Odyssey floating launch platform located in the Pacific Ocean.

In [ ]:
country_dict = {}
for c in countries:
    country_dict[c.name] = c.alpha3
    
data['alpha3'] = data['country']
df = data.replace({"alpha3": country_dict})
df

,organisation,location,date,details,price_in_million,mission_status,rocket_status,year,country,alpha3
0,SpaceX,"SLC-40, Cape Canaveral SFS, Florida",2022-05-14 15:40:00+00:00,Falcon 9 Block 5 | Starlink Group 4-15,"67,000,000.00",Success,Active,2022,USA,USA
1,SpaceX,"SLC-4E, Vandenberg SFB, California",2022-05-13 17:07:00+00:00,Falcon 9 Block 5 | Starlink Group 4-13,"67,000,000.00",Success,Active,2022,USA,USA
2,i-Space,"Site 95, Jiuquan Satellite Launch Center",2022-05-13 02:09:00+00:00,Hyperbola-1 | Jilin-1 Mofang-01A(R),NaN,Failure,Active,2022,China,CHN
3,CASC,"LC-201, Wenchang Satellite Launch Center",2022-05-09 12:56:00+00:00,Long March 7 | Tianzhou 4,NaN,Success,Active,2022,China,CHN
4,SpaceX,"LC-39A, Kennedy Space Center, Florida",2022-05-06 04:42:00+00:00,Falcon 9 Block 5 | Starlink Group 4-17,"67,000,000.00",Success,Active,2022,USA,USA
...,...,...,...,...,...,...,...,...,...,...
6310,US Navy,"LC-18A, Cape Canaveral SFS, Florida",1958-02-04 19:33:00+00:00,Vanguard | Vanguard TV3BU,NaN,Failure,Retired,1958,USA,USA
6311,ABMA,"LC-26A, Cape Canaveral SFS, Florida",1958-01-31 15:47:00+00:00,Juno I | Explorer 1,NaN,Success,Retired,1958,USA,USA
6312,US Navy,"LC-18A, Cape Canaveral SFS, Florida",1957-12-06 04:44:00+00:00,Vanguard | Vanguard TV3,NaN,Failure,Retired,1957,USA,USA
6313,RVSN USSR,"Site 1/5, Baikonur Cosmodrome",1957-11-02 14:30:00+00:00,Sputnik 8K71PS | Sputnik 2,NaN,Success,Retired,1957,Kazakhstan,KAZ


In [ ]:
df.loc[df['country'] == 'Pacific Ocean']

,organisation,location,date,details,price_in_million,mission_status,rocket_status,year,country,alpha3
892,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2014-05-26 16:09:00+00:00,Zenit-3 SL | Eutelsat 3B,NaN,Success,Retired,2014,Pacific Ocean,Pacific Ocean
1002,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2013-02-01 00:55:00+00:00,Zenit-3 SL | Intelsat 27,NaN,Failure,Retired,2013,Pacific Ocean,Pacific Ocean
1013,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2012-12-03 14:43:00+00:00,Zenit-3 SL | Eutelsat-70B,NaN,Success,Retired,2012,Pacific Ocean,Pacific Ocean
1038,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2012-08-19 01:54:00+00:00,Zenit-3 SL | Intelsat 21,NaN,Success,Retired,2012,Pacific Ocean,Pacific Ocean
1053,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2012-06-01 00:22:00+00:00,Zenit-3 SL | Intelsat 19,NaN,Success,Retired,2012,Pacific Ocean,Pacific Ocean
1115,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2011-09-24 15:17:00+00:00,Zenit-3 SL | Atlantic Bird 7,NaN,Success,Retired,2011,Pacific Ocean,Pacific Ocean
1300,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2009-04-20 03:15:00+00:00,Zenit-3 SL | Sicral-1B,NaN,Success,Retired,2009,Pacific Ocean,Pacific Ocean
1344,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2008-09-24 04:27:00+00:00,Zenit-3 SL | Galaxy 19,NaN,Success,Retired,2008,Pacific Ocean,Pacific Ocean
1356,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2008-07-16 00:20:00+00:00,Zenit-3 SL | EchoStar XI,NaN,Success,Retired,2008,Pacific Ocean,Pacific Ocean
1367,Sea Launch,"LP Odyssey, Kiritimati Launch Area",2008-05-21 04:43:00+00:00,Zenit-3 SL | Galaxy 18,NaN,Success,Retired,2008,Pacific Ocean,Pacific Ocean


### Maps

In [ ]:
def plot_map(dataframe, target_column, title, width=800, height=600):
    mapdf = dataframe.groupby(['country', 'alpha3'])[target_column].count().reset_index()
    fig = px.choropleth(
        mapdf, 
        locations="alpha3", 
        hover_name="country", 
        color=target_column, 
        projection="natural earth", 
        title=title,
        color_continuous_scale="pinkyl"
    )
    fig.update_layout(title = title, title_x = 0.5)
    fig.show()

In [ ]:
plot_map(df, 'mission_status', 'Number of starts per country')

In [ ]:
fail_df = df[df['mission_status'] == 'Failure']
plot_map(fail_df, 'mission_status', 'Number of failures per country')

### Analyzing Percent of Successes and Failures for USA and Russia
Since USA and Russia are the 2 countries with the most number of launches, I decided to analyze their data a bit further. With this analysis, we observe that, even if the USA is the number 1 in number of launches, Russia has more succeeded missions than the USA.

In [160]:
# Create dataframe with mission_status by country
df1 = df.groupby(['country', 
                  'mission_status']).size().reset_index(name='counts')

# Create USA dataframe
usa_df = df1[df1['country'] == 'USA'].copy()
del usa_df['country']
usa_df['counts'] = (usa_df['counts'] / 
                    usa_df['counts'].sum()) * 100
usa_df.rename(columns = {'counts':'USA_pct'}, 
              inplace = True)

# Create Russia dataframe
russia_df = df1[df1['country'] == 'Russian Federation'].copy()
del russia_df['country']
russia_df['counts'] = (russia_df['counts'] / 
                       russia_df['counts'].sum()) * 100
russia_df.rename(columns = {'counts':'Russia_pct'}, 
                 inplace = True)

# Merge dataframes
merged_df = usa_df.merge(russia_df, on='mission_status')
merged_df.sort_values('USA_pct', axis = 0, ascending = False)

,mission_status,USA_pct,Russia_pct
3,Success,89.42,94.15
0,Failure,8.50,4.01
1,Partial Failure,2.03,1.78
2,Prelaunch Failure,0.05,0.06


In [178]:
newnames = {'USA_pct':'USA', 'Russia_pct': 'Russia'}

graph = px.bar(merged_df, 
               x='mission_status', 
               y=['USA_pct', 'Russia_pct'], 
              #  color=['USA', 'Russia'],
               height=400,
               color_discrete_map={'USA_pct':'#FF6692',
                                   'Russia_pct':'#19D3F3'},
               labels={'mission_status': 'Status of the Missions',
                     'value': 'Percentage',
                     'variable':'Country'})
graph.update_layout(barmode='group',
                    xaxis={'categoryorder':'total descending'})
graph.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                        legendgroup = newnames[t.name],
                                        hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))

graph.show()

### Summary of the Missions Analyses by Countries and Organisations

In [189]:
df2 = df.groupby(['country', 
                  'organisation', 
                  'mission_status'])['date'].count().reset_index()
df2.rename(columns = {'date':'count'}, inplace = True)

fig = px.sunburst(df2,
                  path=['country', 'organisation', 'mission_status'],
                  values='count',
                  color_discrete_sequence=px.colors.qualitative.Pastel,
                  title='Summary of missions for all countries',
                  width=700,
                  height=700)

fig.show()